### Installing packages

In [1]:
# Importing packages
import os
import sys
import argparse
import re
import json
import Bio
from Bio import SeqIO
from Bio import Entrez
from tqdm import tqdm

In [2]:
# Asserting name of file.
# In fact shortened form of bacteria name - first letter of genus and nine (or lesser) letters of specie.
json_file_name = "M_adhaerens.json"
# Asserting email for Entrez.  Print your own mail in the line below
Entrez.email = "vibrio.choleri.1854@gmail.com"

In [3]:
json_path = f"../../data/jsons/{json_file_name}"
with open(json_path, "r") as json_organism:
    json_organism = json.load(json_organism)

# Creating a variables for futher working with links
organism = json_organism[0]
db_search = "assembly"
db_current = "nucleotide"
pre_complete_ids = list(json_organism[1].values())[0]

In [5]:
def extract_insdc(links): 
    linkset = [ls for ls in links[0]['LinkSetDb'] if
               ls['LinkName'] == 'assembly_nuccore_refseq']
    if 0 != len(linkset):
        uids = [link['Id'] for link in linkset[0]['Link']]
    else:
        uids = 0
    return uids

def download_links(db_search, db_current, complete_id, timer, num_link):
    if timer > 0:
        link_handle = Entrez.elink(dbfrom=db_search, db=db_current, from_uid=complete_id)
        link_record = Entrez.read(link_handle)
        uids = extract_insdc(link_record)
        if uids != 0:
            for uid in uids:
                if uid not in links_checked:  # Checking for duplicates
                    links_checked.append(uid)
                    links.append((uid, num_link))
                    cumulative = 1
                else:
                    cumulative = 0
            num_link += cumulative
    return num_link
            
def forced_download_links(db_search, db_current, complete_id, timer, level, num_link):   # Наверное, это можно реализовать декоратором
    try:
        num_link = download_links(db_search=db_search, db_current=db_current, complete_id=complete_id, timer=timer, num_link=num_link)    
    except RuntimeError:
        print(f"Problem is with {complete_id}")
        timer -= 1
        level += 1
        print(f"We are on the {level} level now")
        num_link = forced_download_links(db_search, db_current, complete_id, timer, level, num_link)
    return num_link

In [7]:
def gcf_transformer(db_search, gcf, level, timer):
    if timer > 0:
        try:
            search_handle = Entrez.esearch(db_search, gcf)
            search_record = Entrez.read(search_handle)
            return search_record
        except RuntimeError:
            print(f"Problem is with {gcf}")
            timer -= 1
            level += 1
            print(f"We are on the {level} level now")
            search_record = gcf_transformer(db_search, gcf, level, timer)

In [8]:
if pre_complete_ids[0][0:3] == "GCF":
    complete_ids = []
    for gcf in tqdm(pre_complete_ids):
        level = 0
        timer = 5 
        search_record = gcf_transformer(db_search, gcf, level, timer)
        complete_ids.append(search_record["IdList"])
else:
    complete_ids = pre_complete_ids

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:57<00:00,  1.18s/it]


In [9]:
# Taking ids for fetching. It collected all non-duplicated links in nucleotide database from assembly database.
# We use try-except for excepting problems with network temorary lags, which ruined our code
links = []
links_checked = []
num_link = 0
for complete_id in tqdm(complete_ids):
    timer = 5
    level = 1
    num_link = forced_download_links(db_search, db_current, complete_id, timer, level, num_link)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:03<00:00,  1.24s/it]


In [14]:
# Collecting data about assemblies
# Fetch with insuring for connection problems
def fetch_insurance(links, db_current, level, timer):
    if timer > 0:
        try:
            gb_handle = Entrez.efetch(db=db_current, rettype="fasta", retmode="text", id=link[0])
            gb_record = SeqIO.read(gb_handle, 'fasta')
            return gb_record, link[1]
        except RuntimeError:
            print(f"Problem is with {gcf}")
            timer -= 1
            level += 1
            print(f"We are on the {level} level now")
            gb_record, link_1 = fetch_insurance(links, db_current, level, timer)

In [16]:
gb_records = []
for link in tqdm(links):
    timer, level = 5, 0
    gb_record, link_1 = fetch_insurance(links, db_current, level, timer)
    gb_records.append((gb_record, link_1))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [33:06<00:00,  5.79s/it]


On the next stage we need to create fasta files for each assembly, to reaanotate them.

In [52]:
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"

In [53]:
dna_type, tuples, source_list = [], [], [] # Creating list for identyfing the number of every assemblie DNA molecules (chromosome and any plasmids)
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
for rec in tqdm(gb_records):
    try:
        source = rec[1] # Number of assembly
        descr = rec[0].description
        if "plasmid" in descr:
            dna_type.append("plasmid")
        elif "scaffold" in descr:
            dna_type.append("scaffold")
        else:
            dna_type.append("chromosome")
        if "scaffold" not in descr: # Для скаффолда иной подход
            source_list.append(source) 
            number = source_list.count(source) # Counting the DNA molecule of assembly
            tuples.append((source, number))
            mask = f"{name}{source}_{number}"
            with open (f"../../{name}/data/for_prokka_fasta/{mask}.fasta", "w") as for_prokka_fasta:
                for_prokka_fasta.write(">")
                for_prokka_fasta.write(mask)
                for_prokka_fasta.write("\n")
                for_prokka_fasta.write(str(rec[0].seq))
                for_prokka_fasta.write("\n")
        elif "scaffold" in descr:
            pass
    except Bio.Seq.UndefinedSequenceError:
        print(rec[0].name)
        continue

  0%|                                                                                                                 | 0/50 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../M_adhaerens/data/for_prokka_fasta/M_adhaerens0_1.fasta'

In [11]:
# Saving plasmid_code
jsonpc1 = json.dumps(tuples)
with open(f"../../{name}/data/{name}_plasmid_code1.json", "w") as jspc1:
    jspc1.write(jsonpc1)
jsonpc2 = json.dumps(dna_type)
with open(f"../../{name}/data/{name}_plasmid_code2.json", "w") as jspc2:
    jspc2.write(jsonpc2)